In [1]:
import csv
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
import spacy
from sklearn.cluster import KMeans
from bs4 import BeautifulSoup
import requests
from collections import defaultdict
from gensim import corpora
import gensim.downloader

# Preprocessing

In [3]:
# create dict for map all abbreviations/shortcuts of states into their full name

res = requests.get("https://abbreviations.yourdictionary.com/articles/state-abbrev.html")
soup = BeautifulSoup(res.text, 'html.parser')
lst = soup.find_all("tr")[1:]
states = [(s.contents[1].string,s.contents[0].string) for s in lst]
states_dict = dict(states)

In [4]:
# create dict for map all abbreviations/shortcuts of countries into their full name

res2 = requests.get('https://www.iban.com/country-codes')
soup = BeautifulSoup(res2.text, 'html.parser')
lst = soup.find_all("tr")[1:]
country = [(c.contents[5].text,c.contents[1].text) for c in lst]
country_dict = dict(country)

In [5]:
# combine the two dict into one

dict_locations = {}
for k,v in states_dict.items():
  dict_locations[k] = v
for k,v in country_dict.items():
  dict_locations[k] = dict_locations.get(k,v)

In [6]:
# stop list for remove stop words from names of columns

res3 = requests.get("https://gist.githubusercontent.com/sebleier/554280/raw/7e0e4a1ce04c2bb7bd41089c9821dbcf6d0c786c/NLTK's%2520list%2520of%2520english%2520stopwords")
stop_list = [word for word in res3.text.split('\n')]

In [16]:
def replacer(string):
    """replace all the abbreviations countries/states
    to full name"""
    
   new_string = ''
   try:
    for n in string.split():
        new_string += f'{dict_locations.get(n,n)} '
    return new_string.strip()
   except Exception:
    return string

In [282]:
def replace_loc(frame):
    for col in frame.columns:
        frame[col] = frame[col].apply(replacer)
    return frame.astype(str)

# The Model

#### Two steps:
#### First, recognize which type the column belongs to: Account/Entity/General/Other,
#### according to the column Name, tf-idf model.

#### Second, recognize which field the column belongs to,
#### according to the tf-idf model with also name entity recognition.

In [160]:
# Make counts per each column and each entity name in the data.
# According to "gensim name entity recognition" model

# ORG - Organization/Company
# GPE - Geographic location
# CARDINAL - Number, Also recognize Address
# MONEY - Belongs to cash: Contribution, Distribution...
# DATE - Dates
# PERSON - Names of persons

class counts:
    
    pca = PCA(n_components=2)
    fields = ['ORG','GPE','CARDINAL','MONEY','DATE','PERSON']
    
    def __init__(self,frame):
        self.frame = frame
        self.nlp = spacy.load("en_core_web_sm")
        self.frequency = []
    
   
    def counts_per_row(self,string):
        doc = self.nlp(string)
        counter = Counter([ent.label_ for ent in doc.ents])
        return np.array([counter.get(k,0) for k in counts.fields])
    
    
    def create_counts(self,frame):
        counts_per_col = []
        for col in frame.columns:
            count = np.zeros(len(counts.fields))
            for words in frame[col]:
                try:
                    count += self.counts_per_row(words)
                except Exception:
                    pass
            counts_per_col.append(count)
        return counts_per_col
    
    # ***optional function***
    def repr_freq(self,freq):
        condition1 = np.where(freq[:,1] + freq[:,2] > 0.5)[0]
        condition2 = np.where((freq[:,0] + freq[:,-1] > 0.8) & (freq[:,0] > freq[:,-1]))[0]
        for i in range(len(freq)):
            if i in condition1:
                if freq[i,2] > 0.2:
                    freq[i,2] = 0.8
                    freq[i,1] = 0.2
                freq[i,[0,-1]] = 0.
            if i in condition2:
                freq[i,0] = 0.8
                freq[i,-1] = 0.2
                freq[i,[1,2]] = 0.
        return freq
    
    def counts_cols_percent(self):
        freq = np.array(self.create_counts(self.frame))
        summation = freq.sum(axis=1).reshape(-1,1)
        summation[summation==0] = 1
        freq = freq/summation
#         freq = self.repr_freq(freq)
        other = (freq > 0.4).sum(axis=1)
        freq = np.concatenate((freq,((other==0).astype(np.int)).reshape(-1,1)),axis=1)
        self.frequency = freq

        
    def plot_dist(self):
        
        dist = counts.pca.fit_transform(self.frequency)
        plt.figure(figsize=(10,10))
        pos = 0.03
        for value,cat in zip(dist, self.frame.columns):
            pos *= -1
            plt.scatter(value[0],value[1] , label=cat)
            plt.annotate(cat, (value[0],value[1]), xytext=(value[0]-0.05, value[1]+pos), 
            arrowprops = dict(arrowstyle="wedge,tail_width=0.5", alpha=0.1), fontsize=15)
        plt.legend(bbox_to_anchor=(1.05, 1), loc=2)
        plt.show()
        
    
    def plot_frequency(self):
        
        for i,col in enumerate(self.frame.columns):
            plt.figure(figsize=(10,10))
            sns.barplot(counts.fields+['OTHER'],self.frequency[i])
            plt.title(col,size=20)
            plt.ylabel('count in %',size=15)
            plt.xlabel('entities',size=15)
            plt.show()
    

### Load the documents from files, which contains the words for the tf-idf,
### and makes lists of the different groups of words

In [106]:
Acount_Names = []
Entity_Names = []
General_Names = []
docs = [Acount_Names,Entity_Names,General_Names]

In [107]:
ORG = []
GPE = []
CARD = []
MONEY = []
DATE = []
PERSON = []
OTHER = []
ent_names = [ORG,GPE,CARD,MONEY,DATE,PERSON,OTHER]

In [108]:
dir = ['Account_Names.txt','Entity_Names.txt','General_Names.txt']
for file,L in zip(dir,docs):
    with open(f'{file}') as f:
        lst = f.readlines()
        for l in lst:
            L.append(l.strip('\n'))

In [109]:
dir = ['ORG.txt','GPE.txt','CARD.txt','MONEY.txt','DATE.txt','PERSON.txt','OTHER.txt']
for file,L in zip(dir,ent_names):
    with open(f'{file}') as f:
        lst = f.readlines()
        for l in lst:
            L.append(l.strip('\n'))

In [110]:
def strip_punc(lst):
    """remove punctuations from the column names"""
    new_list = []
    for s in lst:
        for word in re.split('_|=|-|:|,|/|[(]|[)]| ',s.lower()):
            if word not in stop_list:
                new_list.append(word)
    return new_list
        

In [111]:
# Create similarity index of the types of the columns(Account,Entity,General)
# in order to check the similarity between each column name and the types

types_names_docs = [strip_punc(doc) for doc in docs]
types_dictionary = gensim.corpora.Dictionary(types_names_docs)
types_corpus = [types_dictionary.doc2bow(gen_doc) for gen_doc in types_names_docs]
types_tf_idf = gensim.models.TfidfModel(types_corpus)
sims = gensim.similarities.Similarity('/content',types_tf_idf[types_corpus],
                                        num_features=len(types_dictionary))

In [112]:
# Create similarity index of the fields of the columns('ORG','GPE','CARDINAL','MONEY','DATE','PERSON')
# in order to check the similarity between each column name and the field

entities = [strip_punc(doc) for doc in ent_names]
ent_dictionary = gensim.corpora.Dictionary(entities)
ent_corpus = [ent_dictionary.doc2bow(gen_doc) for gen_doc in entities]
tf_idf_ent = gensim.models.TfidfModel(ent_corpus)
ent_sims = gensim.similarities.Similarity('/content',tf_idf_ent[ent_corpus],
                                        num_features=len(ent_dictionary))


In [125]:
def get_column_type(frame):
    '''return the indexes of the columns per each type'''
    
    corpus = [types_dictionary.doc2bow(re.split('_|=|-|:|,| ',col.lower())) for col in frame.columns]
    groups = sims[types_tf_idf[corpus]]
    other = ((groups.sum(axis=1)==0).astype(int)).reshape(-1,1)
    cols_type = (np.concatenate((groups,other),axis=1)) + 1
    account_cols = np.where(np.argmax(cols_type,axis=1)==0)[0]
    entity_cols = np.where(np.argmax(cols_type,axis=1)==1)[0]
    general_cols = np.where(np.argmax(cols_type,axis=1)==2)[0]
    other_cols = np.where(np.argmax(cols_type,axis=1)==3)[0]
    return account_cols,entity_cols,general_cols,other_cols

In [198]:
def get_entities(frame):
    '''return the indexes of the columns per each field'''
    
    corpus = [ent_dictionary.doc2bow(re.split('_|=|-|:|,| ',col.lower())) for col in frame.columns]
    groups = ent_sims[tf_idf_ent[corpus]] + 1
    count = counts(frame)
    count.counts_cols_percent()
    frequencies = count.frequency + 1
    entity_freq = (frequencies)*(9**(groups))
    ORG_cols = np.where(entity_freq.argmax(axis=1)==0)[0]
    GPE_cols = np.where(entity_freq.argmax(axis=1)==1)[0]
    CARDINAL_cols = np.where(entity_freq.argmax(axis=1)==2)[0]
    MONEY_cols = np.where(entity_freq.argmax(axis=1)==3)[0]
    DATE_cols = np.where(entity_freq.argmax(axis=1)==4)[0]
    PERSON_cols = np.where(entity_freq.argmax(axis=1)==5)[0]
    OTHER_cols = np.where(entity_freq.argmax(axis=1)==6)[0]
    
    return ORG_cols,GPE_cols,CARDINAL_cols,MONEY_cols,DATE_cols,PERSON_cols,OTHER_cols

In [235]:
def get_predict(frame):
    '''print for each column in the data the predicted
        type and the field'''
    
    types = [get_column_type(frame)]
    fields = [get_entities(frame)]
    
    types_names = ['Account','Entity','General','OTHER']
    fields_names = ['ORG', 'GPE', 'CARDINAL', 'MONEY', 'DATE', 'PERSON', 'OTHER']
    
    idx_type = {}
    idx_field = {}
    
    for name,group in zip(types_names,types[0]):
        for id in group:
            idx_type[id] = name
    for name,group in zip(fields_names,fields[0]):
        for id in group:
            idx_field[id] = name
            
    for i,col in enumerate(frame.columns):
        print(f'column name: {col},  predict: {idx_type[i], idx_field[i]}\n')
            